In [64]:
import requests
from decimal import Decimal


correct = Decimal('3754.8057')
result = convert(Decimal("1000.1000"), 'RUR', 'JPY', "17/02/2005", requests)
if result == correct:
    print("Correct")
else:
    print("Incorrect: %s != %s" % (result, correct))

Correct


In [ ]:
В этом задании придется написать свой конвертер валют (см. приложенный файл). 
Курсы валют нужно брать из API Центробанка. Документация по нему тут, потребуется только XML_daily.asp. 
(Обратите внимание, что указанный в документации API протокол http больше не поддерживается при запросах 
 с помощью requests.get, указывайте в строке запроса https)  

В функцию convert(amount, cur_from, cur_to, date, requests) будет передана сумма amount в валюте с кодом cur_from, 
и её требуется перевести в валюту cur_to через рубль (код: RUR). Для запроса к API нужно использовать переданный requests, 
точнее, его метод get().

Все суммы и курсы требуется хранить в Decimal, т.к. для финансовых данных вычисления с фиксированной точкой подходят больше.

Конечный результат нужно округлить до 4-х знаков, перед тем как вернуть его из функции. Посмотрите метод quantize().

Для некоторых валют курс возвращается из расчета не на одну денежную единицу указанной валюты, а на 10 или даже 100, 
поэтому у курса валюты в XML есть не только Value, но и Nominal, и справедлива формула: Nominal ед. валюты = Value рублей.

При проверке на сервере сеть недоступна. В функцию будет передан фейковый requests, его интерфейс и response аналогичны 
настоящему. Если его использовать в объеме, требуемом для задания, разницы не будет заметно.

In [63]:
from bs4 import BeautifulSoup
from decimal import Decimal


def convert(amount, cur_from, cur_to, date, requests):
    response = requests.get(f'http://cbr.ru/scripts/XML_daily.asp?date_req={date}')
    soup = BeautifulSoup(response.content, 'xml')
    rate = str(soup.find('CharCode', text=cur_to).find_next_sibling('Value').string)
    rate = Decimal(rate.replace(',','.'))
    result = ((amount / rate) * 100).quantize(Decimal('.0001'))
    return result  
